In [ ]:
import json
import pandas as pd
import seaborn as sns
from datetime import datetime

In [ ]:
data = []
with open("recordtest_2024-10-30_17-44-39.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

In [ ]:
data[0]

In [ ]:
for d in data:
    for value, key in enumerate(d["data"]):
        d[key] = d["data"][key]
    del d["data"]
data[0]

In [ ]:
data[1]["timestamp"]

In [ ]:
# Example timestamps
timestamp1 = "2024-10-25T11:20:03.294735"
timestamp2 = "2024-10-25T11:20:03.315138"

# Convert strings to datetime objects
time1 = datetime.fromisoformat(timestamp1)
time2 = datetime.fromisoformat(timestamp2)

# Calculate the time difference
time_difference = time2 - time1

# Display the time difference
print("Time difference:", time_difference)
print("Total seconds:", time_difference.total_seconds())

In [ ]:
stretch_data = [x for x in data if x["channel"] == "stretch_status"]
quest_data = [x for x in data if x["channel"] == "quest_control"]

print(len(stretch_data))
print(len(quest_data))

In [ ]:
for i in range(len(stretch_data) - 1, 0, -1):
    curr = data[i]
    prev = data[i - 1]
    curr_time = datetime.fromisoformat(curr["timestamp"])
    prev_time = datetime.fromisoformat(prev["timestamp"])

    delta = (prev_time - curr_time).total_seconds()

    data[i]["x_velocity"] = (curr["x"] - prev["x"]) / delta
    data[i]["y_velocity"] = (curr["y"] - prev["y"]) / delta
    data[i]["arm_velocity"] = (curr["arm"] - prev["arm"]) / delta
    data[i]["lift_velocity"] = (curr["lift"] - prev["lift"]) / delta

    data[i]["theta_velocity"] = (curr["theta"] - prev["theta"]) / delta
    data[i]["roll_velocity"] = (curr["wrist_roll"] - prev["wrist_roll"]) / delta
    data[i]["yaw_velocity"] = (curr["wrist_yaw"] - prev["wrist_yaw"]) / delta
    data[i]["pitch_velocity"] = (curr["wrist_pitch"] - prev["wrist_pitch"]) / delta
    data[i]["head_tilt_velocity"] = (curr["head_tilt"] - prev["head_tilt"]) / delta
    data[i]["head_pan_velocity"] = (curr["head_pan"] - prev["head_pan"]) / delta
    data[i]["gripper_velocity"] = (
        curr["stretch_gripper"] - prev["stretch_gripper"]
    ) / delta

In [ ]:
for i in range(len(quest_data) - 1, 0, -1):
    curr = data[i]
    prev = data[i - 1]
    curr_time = datetime.fromisoformat(curr["timestamp"])
    prev_time = datetime.fromisoformat(prev["timestamp"])

    delta = (prev_time - curr_time).total_seconds()

    data[i]["x_velocity"] = (curr["x"] - prev["x"]) / delta
    data[i]["y_velocity"] = (curr["y"] - prev["y"]) / delta
    data[i]["arm_velocity"] = (curr["arm"] - prev["arm"]) / delta
    data[i]["lift_velocity"] = (curr["lift"] - prev["lift"]) / delta

    data[i]["theta_velocity"] = (curr["theta"] - prev["theta"]) / delta
    data[i]["roll_velocity"] = (curr["wrist_roll"] - prev["wrist_roll"]) / delta
    data[i]["yaw_velocity"] = (curr["wrist_yaw"] - prev["wrist_yaw"]) / delta
    data[i]["pitch_velocity"] = (curr["wrist_pitch"] - prev["wrist_pitch"]) / delta
    data[i]["head_tilt_velocity"] = (curr["head_tilt"] - prev["head_tilt"]) / delta
    data[i]["head_pan_velocity"] = (curr["head_pan"] - prev["head_pan"]) / delta
    data[i]["gripper_velocity"] = (
        curr["stretch_gripper"] - prev["stretch_gripper"]
    ) / delta

In [ ]:
df = pd.DataFrame(stretch_data + quest_data)
df["timestamp"] = df["timestamp"].apply(lambda x: datetime.fromisoformat(x))

In [ ]:
df

In [ ]:
velocity_columns = [col for col in df.columns if "velocity" in col]
df_velocity = df.melt(
    id_vars=["timestamp", "channel", "data_type"],  # Keep these columns as identifiers
    value_vars=velocity_columns,  # Columns to be transformed into rows
    var_name="type",  # New column name for 'velocity' type
    value_name="velocity",  # New column name for the velocity values
)

# Step 3: Clean the 'type' column to only reflect the velocity type (remove '_velocity' suffix)
df_velocity["type"] = df_velocity["type"].str.replace("_velocity", "")

# Display the result
print(df_velocity.head())

In [ ]:
df_velocity.columns
df_velocity_filtered = df_velocity[
    df_velocity["timestamp"] >= datetime.fromisoformat("2024-10-25T11:23:06.353725")
]
df_velocity_filtered = df_velocity_filtered[
    df_velocity_filtered["timestamp"]
    <= datetime.fromisoformat("2024-10-25T11:23:27.114610")
]

In [ ]:
sns.lineplot(
    data=df_velocity_filtered, x="timestamp", y="velocity", style="channel", hue="type"
)

In [ ]:
sns.lineplot(
    data=df_velocity_filtered[df_velocity_filtered["lift"] == "theta"],
    x="timestamp",
    y="velocity",
    hue="channel",
)